# AM207 Project: Mastermind

### Expectation Maximization
For this project we also tried to deploy expectation maximization (EM) to solve for the hidden code. At first sight this may seem very intuitive, since the parameters of the problem—the colors, or numbers, in the hidden code—are unobserved. Treating this like a latent parameter, like on does in expectation-maximization may seem like a good approach to take.

Trying to work out the mathematics of a Mastermind expectation maximization strategy yields the following. EM is typically used in problems where the likelihood is hard to evaluate directly. Therefore, we recursively maximize the expectation of the likelihood conditional on the data, until we converge to a set of parameters that is close to the maximum likelihood estimator. In Mastermind, however, the likelihood of the data given a set of parameters is not so hard to evaluate at all. Since the constraint on the problem is that the hidden code should logically allow for the number of black and white pegs appearing given a certain guess, there can be many hidden codes with a likelihood of 1, while the rest will have likelihood 0. For example, if my guess is 1234, and as a response I get back two black pegs, then I know two numbers are right. Any hidden code where two numbers are in the same position and the rest are not, could validly produce this outcome, e.g. the code 1255 or 1664, etcetera. Given that such a code is true, the likelihood that the response was two black pegs is 1, since this is deterministically produced by the rules of the game. Therefore, there will be a split in the solution space between codes that are possible given the responses (likelihood equals 1), and codes that are impossible (likelihood equals 0). EM, in this case, would then converge to some solution in the set of still allowable codes. Since the choice of a new code isn't directed by any possible information gains, this amounts to a random search across the space. If we make an informed decision here, the problem boils down to a strategy where one would maximize the information gain, for instance using entropy or some other metric.

In conclusion, EM is probably not a good way to study the Mastermind problem, since the game is underconstrained and deterministic, leading to a space of many equally likely convergent solutions at any given round in the game. It may be more useful in estimating parameters of stochastic processes with a lot of data, rather than in making forward looking decisions where information gain plays a bigger role.

In [54]:
from MMboard import MMboard 
import numpy as np
import itertools

In [206]:
# Expectation Maximization, but more like maximum likelihood actually

def expectation_maximization(n_code, n_colors, code=None, silent=True):
    
    # Get board ready to play
    board = MMboard(codelength=n_code, numcolors=n_colors, suppress_output=silent)
    board.set_code() if not code else board.set_code(code)
    
    # Value stores
    solutions = np.array(list(itertools.product(range(n_colors), repeat=n_code)))
    guesses = []
    responses = []
    
    # Emulates response given some hidden code
    def response(guess, code):
        equals = np.array(map(lambda (a, b): a == b, zip(guess, code)))
        numbers = np.unique(guess[equals == False])
        found_places = np.array(map(lambda a: np.count_nonzero(code[equals == False] == a), numbers))
        occurrences = np.array(map(lambda a: np.count_nonzero(guess[equals == False] == a), numbers))
        wrong_places = np.array(map(lambda (a, b): a if a <= b else b, zip(found_places, occurrences)))
        return len(equals.nonzero()[0]), len(wrong_places.nonzero()[0])
    
    # Likelihood defined as above
    likelihood = lambda guess, outcome, code: 1 if response(guess, code) == outcome else 0
    
    # Run as long we're not game over
    while not board.gameover:
        
        # Make an informed guess from set of solutions with maximum likelihood
        guess = solutions[np.random.randint(0, len(solutions))]
        outcome = board.guess_code(guess)
        guesses.append(guess)
        responses.append(outcome)
        
        # Compute feasible solutions left
        valid_indices = []
        for i in xrange(len(solutions)):
            solution = solutions[i]
            total_likelihood = 1
            
            # Compute likelihood over past outcomes
            for j in xrange(len(guesses)):
                guess = guesses[j]
                outcome = responses[j]
                if likelihood(guess, outcome, solution) == 0:
                    total_likelihood = 0
                    break
            
            # Keep only solutions with likelihood > 0
            if total_likelihood > 0:
                valid_indices.append(i)
        
        # Set solutions
        solutions = solutions[valid_indices]

In [207]:
expectation_maximization(4, 6, silent=False)

Code successfully initialized to  [5 1 4 0] 

guess #1 of 10: you guessed  [4 0 3 3]
You have 0 right item(s) in the right place, and
  2 right item(s) but in the wrong place

guess #2 of 10: you guessed  [0 4 5 5]
You have 0 right item(s) in the right place, and
  3 right item(s) but in the wrong place

guess #3 of 10: you guessed  [1 5 4 0]
You have 2 right item(s) in the right place, and
  2 right item(s) but in the wrong place

guess #4 of 10: you guessed  [5 1 4 0]
You have 4 right item(s) in the right place
You win!
